In [13]:
import pandas as pd
import re

# ========= 1. 路径 =========
left_fp = r"F:/00.博后研究/城乡连续体制图/New_NEW/indexes_of_30667_towns_derived_from_imageries.csv"
right_fp = r"F:/00.博后研究/城乡连续体制图/New_NEW/全国乡镇.csv"

# ========= 2. 读数据 =========
left = pd.read_csv(left_fp, encoding="utf-8")
right = pd.read_csv(right_fp, encoding="utf-8")

# 左表主要字段去空格
for col in ["Province", "City", "COUNTY", "Town"]:
    left[col] = left[col].astype(str).str.strip()

# 左表代码列清洗
for col in ["Province_code", "City_code", "County_code", "Town_code"]:
    if col in left.columns:
        left[col] = (
            left[col].astype(str)
            .str.replace(r"\.0$", "", regex=True)
            .str.strip()
        )

# 右表名字去空格
for col in ["省级", "市级", "区县级", "乡镇级"]:
    right[col] = right[col].astype(str).str.strip()

# ========= 3. 把右表代码统一成6位 =========
def to_digits(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    digits = re.sub(r"\D", "", s)
    return digits

def to_six(x):
    d = to_digits(x)
    return d[:6] if len(d) >= 6 else d

right["省代码_6"] = right["省代码"].apply(to_six)
right["市代码_6"] = right["市代码"].apply(to_six)
right["区县码_6"] = right["区县码"].apply(to_six)
right["乡镇码_6"] = right["乡镇码"].apply(to_digits)

# ========= 4. 名称标准化 =========
def norm_name(x: str):
    if pd.isna(x):
        return ""
    x = str(x).strip()
    x = re.sub(r"\s+", "", x)
    tails = [
        "街道办事处", "街道办", "街道",
        "镇人民政府", "乡人民政府",
        "镇政府", "乡政府",
        "管理委员会", "管理区", "管委会",
        "社区居委会", "居民委员会",
        "办事处",
        "村委会",
        "镇", "乡"
    ]
    for t in tails:
        if x.endswith(t):
            x = x[:-len(t)]
            break
    return x

left["Town_norm"] = left["Town"].apply(norm_name)

# 右表取需要的字段并也加标准化
right_use = right[
    ["FID", "省级", "市级", "区县级", "乡镇级",
     "省代码_6", "市代码_6", "区县码_6", "乡镇码_6"]
].copy()
right_use["乡镇级_norm"] = right_use["乡镇级"].apply(norm_name)
right_use["市级_norm"] = right_use["市级"].apply(norm_name)
right_use["区县级_norm"] = right_use["区县级"].apply(norm_name)

# ========= 5. 第1轮：县码 + 原始镇名 =========
m1 = left.merge(
    right_use,
    how="left",
    left_on=["County_code", "Town"],
    right_on=["区县码_6", "乡镇级"],
    suffixes=("", "_nat")
)

m1_matched_mask = m1["FID"].notna()
print("第1轮匹配成功:", m1_matched_mask.sum())

# ========= 6. 第2轮：县码 + 标准化镇名（对第1轮未匹配部分） =========
# 取出第1轮没配上的，给它一个 row_id 用来回写
left_un1 = m1[~m1_matched_mask].copy()
left_un1 = left_un1.reset_index(drop=False).rename(columns={"index": "row_id_m1"})

m2 = left_un1.merge(
    right_use,
    how="left",
    left_on=["County_code", "Town_norm"],
    right_on=["区县码_6", "乡镇级_norm"],
    suffixes=("", "_nat2")
)

m2_matched = m2[m2["FID"].notna()]
print("第2轮匹配成功:", len(m2_matched))

# 把第2轮结果写回 m1
for _, r in m2_matched.iterrows():
    idx = r["row_id_m1"]  # 这是 m1 里的原始行号
    m1.loc[idx, "FID"] = r["FID"]
    m1.loc[idx, "省级"] = r["省级"]
    m1.loc[idx, "市级"] = r["市级"]
    m1.loc[idx, "区县级"] = r["区县级"]
    m1.loc[idx, "乡镇级"] = r["乡镇级"]
    m1.loc[idx, "省代码_6"] = r["省代码_6"]
    m1.loc[idx, "市代码_6"] = r["市代码_6"]
    m1.loc[idx, "区县码_6"] = r["区县码_6"]
    m1.loc[idx, "乡镇码_6"] = r["乡镇码_6"]

# ========= 7. 第3轮：市码 + 标准化镇名（兜底） =========
m1_un2 = m1[m1["FID"].isna()].copy()
m1_un2 = m1_un2.reset_index(drop=False).rename(columns={"index": "row_id_m1"})

m3 = m1_un2.merge(
    right_use,
    how="left",
    left_on=["City_code", "Town_norm"],
    right_on=["市代码_6", "乡镇级_norm"],
    suffixes=("", "_nat3")
)

m3_matched = m3[m3["FID"].notna()]
print("第3轮匹配成功:", len(m3_matched))

for _, r in m3_matched.iterrows():
    idx = r["row_id_m1"]
    m1.loc[idx, "FID"] = r["FID"]
    m1.loc[idx, "省级"] = r["省级"]
    m1.loc[idx, "市级"] = r["市级"]
    m1.loc[idx, "区县级"] = r["区县级"]
    m1.loc[idx, "乡镇级"] = r["乡镇级"]
    m1.loc[idx, "省代码_6"] = r["省代码_6"]
    m1.loc[idx, "市代码_6"] = r["市代码_6"]
    m1.loc[idx, "区县码_6"] = r["区县码_6"]
    m1.loc[idx, "乡镇码_6"] = r["乡镇码_6"]

# ========= 8. 导出 =========
out_all = r"F:/00.博后研究/城乡连续体制图/New_NEW/indexes_of_30667_towns_with_natcodes_bycode_name.csv"
m1.to_csv(out_all, index=False, encoding="utf-8-sig")

still_unmatched = m1[m1["FID"].isna()].copy()
out_unmatched = r"F:/00.博后研究/城乡连续体制图/New_NEW/indexes_of_30667_towns_unmatched_bycode_name.csv"
still_unmatched.to_csv(out_unmatched, index=False, encoding="utf-8-sig")

print("最终未匹配数量:", still_unmatched.shape[0])
print("结果已导出:", out_all)
print("未匹配已导出:", out_unmatched)


第1轮匹配成功: 29380
第2轮匹配成功: 0
第3轮匹配成功: 0
最终未匹配数量: 1288
结果已导出: F:/00.博后研究/城乡连续体制图/New_NEW/indexes_of_30667_towns_with_natcodes_bycode_name.csv
未匹配已导出: F:/00.博后研究/城乡连续体制图/New_NEW/indexes_of_30667_towns_unmatched_bycode_name.csv
